In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook

In [2]:
#data = body of post request will all values of first and third picker + 1st value of 2nd picker (use postman)
with(open('post_request_body.txt', "rb")) as f:
    body = [tuple(x.decode().strip().split(':')) for x in f.readlines()]

In [3]:
def find_idx_for_key(body, key):
    for idx, item in enumerate(body): 
        if item[0] == key: 
            return idx

In [4]:
#find index of the variable in the body
key = 'ctl00$ContentPlaceHolderMain$VariableSelector1$VariableSelector1$VariableSelectorValueSelectRepeater$ctl02$VariableValueSelect$VariableValueSelect$ValuesListBox'
idx_key = find_idx_for_key(body, key)

In [5]:
#find all options for second picker
URL = "https://www.pxweb.bfs.admin.ch/Selection.aspx?px_language=fr&px_db=px-x-1703030000_101&px_tableid=px-x-1703030000_101/px-x-1703030000_101.px&px_type=PX"
soup = BeautifulSoup(requests.get(URL).text, 'html.parser')
options = [o.attrs['value'] for o in soup.select('#ctl00_ContentPlaceHolderMain_VariableSelector1_VariableSelector1_VariableSelectorValueSelectRepeater_ctl02_VariableValueSelect_VariableValueSelect_ValuesListBox option')]
len(options)

312

In [6]:
# GET, params saving
r = requests.get(URL).text
soup = BeautifulSoup(r, 'html.parser')
body[find_idx_for_key(body, '__EVENTVALIDATION')] = ('__EVENTVALIDATION', soup.find(id="__EVENTVALIDATION").attrs['value'])
body[find_idx_for_key(body, '__VIEWSTATE')] = ('__VIEWSTATE', soup.find(id="__VIEWSTATE").attrs['value'])
body[find_idx_for_key(body, '__VIEWSTATEGENERATOR')] = ('__VIEWSTATEGENERATOR', soup.find(id="__VIEWSTATEGENERATOR").attrs['value'])

In [7]:
#Note URL_post != URL
URL_post = 'https://www.pxweb.bfs.admin.ch/Selection.aspx?px_language=fr&px_db=px-x-1703030000_101&px_tableid=px-x-1703030000_101%2fpx-x-1703030000_101.px&px_type=PX'

df = pd.DataFrame()

for o in tqdm_notebook(options):
    
    #Change the value of the variable in the body of the post request
    body[idx_key] = (key, o)
    
    #Make the post request
    r = requests.post(URL_post, data=body)
    
    #Build the temporary dataframe corresponding to one votation
    df_tmp = pd.read_html(r.text, thousands=' ', decimal=',', na_values=['...'])[0]
    df_tmp.columns = df_tmp.columns.droplevel()
    df_tmp.columns = ['Commune', 'Votation'] + df_tmp.columns[1:-1].tolist()
    
    #Concat with the temporary dataframe with the dataframe containing all the votations
    df = pd.concat([df, df_tmp])
    
df = df.sort_index()
df = df.set_index(['Commune', 'Votation'])

df

KeyboardInterrupt: 

In [ ]:
df.to_pickle("data/votations.pkl")